In [3]:
# Import packages
import os
import rasterio
from rasterio.merge import merge
from rasterio.enums import Resampling
import geopandas as gpd
import numpy as np
from rasterio.warp import calculate_default_transform, reproject
import subprocess
from rasterio.mask import mask
import pandas as pd


In [4]:
# Merge WSF files

wsf_folder = r"C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\World_Settlement_Footprint"

# gather all .tif paths
wsf_files = [
    os.path.join(wsf_folder, f)
    for f in os.listdir(wsf_folder)
    if f.lower().endswith('.tif')
]

# 3. Define output path
out_path = os.path.join(wsf_folder, "WSF_merged_31467.tif")

# 4. Build the gdalwarp command:
#    - -multi + NUM_THREADS=ALL_CPUS for multithreading
#    - -r nearest to preserve 0/1 values
#    - -t_srs EPSG:31467 target CRS
#    - -tr 10 10 target resolution (10 m)
cmd = [
    "gdalwarp",
    "-multi",
    "-wo", "NUM_THREADS=ALL_CPUS",
    "-r", "near",
    "-t_srs", "EPSG:31467",
    "-tr", "10", "10",
] + wsf_files + [out_path]

# 5. Run it
print("Running:", " ".join(cmd))
subprocess.run(cmd, check=True)

print("Merged & reprojected raster written to:", out_path)


Running: gdalwarp -multi -wo NUM_THREADS=ALL_CPUS -r near -t_srs EPSG:31467 -tr 10 10 C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\World_Settlement_Footprint\WSF2015_Germany-0000000000-0000000000.tif C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\World_Settlement_Footprint\WSF2015_Germany-0000065536-0000000000.tif C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\World_Settlement_Footprint\WSF_merged_31467.tif C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\World_Settlement_Footprint\WSF_merged_31467.tif


CalledProcessError: Command '['gdalwarp', '-multi', '-wo', 'NUM_THREADS=ALL_CPUS', '-r', 'near', '-t_srs', 'EPSG:31467', '-tr', '10', '10', 'C:\\Users\\agnes\\Documents\\EAGLE\\Innovation_Lab\\Daten\\World_Settlement_Footprint\\WSF2015_Germany-0000000000-0000000000.tif', 'C:\\Users\\agnes\\Documents\\EAGLE\\Innovation_Lab\\Daten\\World_Settlement_Footprint\\WSF2015_Germany-0000065536-0000000000.tif', 'C:\\Users\\agnes\\Documents\\EAGLE\\Innovation_Lab\\Daten\\World_Settlement_Footprint\\WSF_merged_31467.tif', 'C:\\Users\\agnes\\Documents\\EAGLE\\Innovation_Lab\\Daten\\World_Settlement_Footprint\\WSF_merged_31467.tif']' returned non-zero exit status 1.

In [5]:
# Files
shp_file = r"C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\Verwaltungsgebiete\vg250_12-31.gk3.shape.ebenen\vg250_ebenen_1231\VG250_KRS.shp"
gdf = gpd.read_file(shp_file)
wsf_raster  = "C:/Users/agnes/Documents/EAGLE/Innovation_Lab/Daten/World_Settlement_Footprint/WSF_merged_31467.tif"

# Definiere die Pixelgröße (10m x 10m)
pixel_area = 10 * 10  # 10m * 10m = 100 m² pro Pixel

first_polygon = gdf.iloc[0]  # Nur der erste (Land-)kreis (Polygon)

with rasterio.open(wsf_raster) as src:
    # Maske erstellen für den ersten Kreis
    geom = [first_polygon.geometry]
    out_image, _ = mask(src, geom, crop=True, filled=False)
    arr = out_image[0] 

    # Nur gültige (nicht-masked) Pixel analysieren
    valid_pixels = arr.compressed()  # Maskierte Pixel entfernen
    total_pixels = valid_pixels.size

    if total_pixels == 0:
        settlement_area = 0.0
        total_area = 0.0
        pct = 0.0
    else:
        # Zähle alle Pixel mit einem Wert > 0 als Siedlungsfläche
        settlement_pixels = np.sum(valid_pixels > 0)
        settlement_area = settlement_pixels * pixel_area
        total_area = total_pixels * pixel_area
        pct = (settlement_area / total_area) * 100

    # Ergebnis anzeigen
    print(f"AGS: {first_polygon['AGS']}")
    print(f"Name: {first_polygon['GEN']}")
    print(f"Settlement Area: {settlement_area} m²")
    print(f"Total Area: {total_area} m²")
    print(f"Settlement Coverage: {pct:.2f} %")


AGS: 01001
Name: Flensburg
Settlement Area: 23701700 m²
Total Area: 49286400 m²
Settlement Coverage: 48.09 %


In [6]:
# Paths 
shp_file = r"C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\Verwaltungsgebiete\vg250_12-31.gk3.shape.ebenen\vg250_ebenen_1231\VG250_KRS.shp"
wsf_raster  = r"C:/Users/agnes/Documents/EAGLE/Innovation_Lab/Daten/World_Settlement_Footprint/WSF_merged_31467.tif"
output_csv  = r"C:/Users/agnes/Documents/EAGLE/Innovation_Lab/Daten/settlement_coverage_by_district.csv"

gdf = gpd.read_file(shp_file)

# Open the WSF raster to grab its CRS and pixel size
with rasterio.open(wsf_raster) as src:
    raster_crs   = src.crs
    pixel_area   = abs(src.res[0] * src.res[1])  # m² per pixel

# Reproject districts to the raster CRS if needed
if gdf.crs != raster_crs:
    gdf = gdf.to_crs(raster_crs)

# Loop over each district, mask & compute coverage
results = []
with rasterio.open(wsf_raster) as src:
    for _, row in gdf.iterrows():
        geom = [row.geometry]
        out_image, _ = mask(src, geom, crop=True, filled=False)
        arr = out_image[0]  # masked array

        # only pixels inside the polygon
        valid_pixels = arr.compressed()
        total_pixels = valid_pixels.size

        if total_pixels == 0:
            settlement_area = 0.0
            total_area      = 0.0
            pct             = 0.0
        else:
            # count any positive value as settlement
            settlement_pixels = np.sum(valid_pixels > 0)
            settlement_area   = settlement_pixels * pixel_area
            total_area        = total_pixels * pixel_area
            pct               = (settlement_area / total_area) * 100

        results.append({
            'AGS': row.get('AGS'),
            'Name': row.get('GEN'),
            'Settlement_Area_m2': settlement_area,
            'Total_Area_m2':    total_area,
            'Coverage_Percent': pct
        })
        
# Save results to CSV
df = pd.DataFrame(results)
df.to_csv(output_csv, index=False)
print("Done! Results written to:", output_csv)


Done! Results written to: C:/Users/agnes/Documents/EAGLE/Innovation_Lab/Daten/settlement_coverage_by_district.csv
